In [1]:
from tf.app import use
import os
import collections
from itertools import chain
import re
import textwrap

A = use('bhsa', hoist=globals())

%load_ext autoreload
%autoreload 2

This is Text-Fabric 9.1.1
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

122 features found and 0 ignored


In [2]:
# Random lex data

lex_dict = collections.defaultdict(int)
g_lex_dict = collections.defaultdict(int)
g_lex_utf8_dict = collections.defaultdict(int)

# Collect all nodes of type 'word'
for word in F.otype.s('word'):
    lex = F.lex.v(word)
    g_lex = F.g_lex.v(word)
    g_lex_utf8 = F.g_lex_utf8.v(word)

    lex_dict[lex] += 1
    g_lex_dict[g_lex] += 1
    g_lex_utf8_dict[g_lex_utf8] += 1

for word in F.otype.s('word'):
    lex = F.lex.v(word)
    g_lex = F.g_lex.v(word)
    g_lex_utf8 = F.g_lex_utf8.v(word)
    a = lex_dict[lex]
    b = g_lex_dict[g_lex]
    c = g_lex_utf8_dict[g_lex_utf8]
    d = F.freq_lex.v(word)
    cons = F.g_cons.v(word)
    print(f"{word}: {cons}\nlex: {a}\ng_lex: {b}\nutf8: {c}\nfreq: {d}")

1: B
lex: 15768
g_lex: 6588
utf8: 6588
freq: 15542
2: R>CJT
lex: 51
g_lex: 49
utf8: 49
freq: 51
3: BR>
lex: 48
g_lex: 9
utf8: 9
freq: 48
4: >LHJM
lex: 2601
g_lex: 2389
utf8: 2389
freq: 2601
5: >T
lex: 10987
g_lex: 1171
utf8: 1171
freq: 10987
6: H
lex: 30392
g_lex: 17008
utf8: 17008
freq: 30386
7: CMJM
lex: 421
g_lex: 225
utf8: 225
freq: 421
8: W
lex: 51003
g_lex: 24303
utf8: 24303
freq: 50272
9: >T
lex: 10987
g_lex: 1171
utf8: 1171
freq: 10987
10: H
lex: 30392
g_lex: 6633
utf8: 6633
freq: 30386
11: >RY
lex: 2504
g_lex: 1350
utf8: 1350
freq: 2504
12: W
lex: 51003
g_lex: 24303
utf8: 24303
freq: 50272
13: H
lex: 30392
g_lex: 6633
utf8: 6633
freq: 30386
14: >RY
lex: 2504
g_lex: 1350
utf8: 1350
freq: 2504
15: HJTH
lex: 3561
g_lex: 480
utf8: 480
freq: 3561
16: THW
lex: 20
g_lex: 11
utf8: 11
freq: 20
17: W
lex: 51003
g_lex: 1304
utf8: 1304
freq: 50272
18: BHW
lex: 3
g_lex: 3
utf8: 3
freq: 3
19: W
lex: 51003
g_lex: 24303
utf8: 24303
freq: 50272
20: XCK
lex: 80
g_lex: 78
utf8: 78
freq: 80
21: <

KeyboardInterrupt: 

In [3]:
class Classify:
    
    def __init__(self):
        self.stop_words = ['prep', 'art', 'conj']
        self.scale = {
            1:list(range(500, 50000)),  # frequent
            2:list(range(250, 500)),    # common
            3:list(range(150, 250)),    # medium
            5:list(range(50, 150)),     # uncommon
            8:list(range(1, 50))        # rare
        }

In [4]:
class Paragraph:

    def __init__(self, id = 0):
        self.id = id
        self.verses = []
        self.weight = 0
        self._type = ''
        self.types = {'פ': 'open', 'ס': 'closed'}

    def get_all_words(self):
        words = []
        for verse in self.verses:
            for word in L.i(verse, otype='word'):
                words.append(word)
        return words

    def get_vs_words(self, verse):
        words = [w for w in L.i(verse, otype='word')]
        return words

    def get_text(self):
        return T.text(self.verses, fmt='text-orig-full')
    
    def reset_values(self):
        self.verses = []
        self.score = 0
        self.type = ''

    def get_vs_weights(self):
        # A class with weight scale data.
        c = Classify()
        verse_weights = {}
        # Iterate over verses in the paragraph.
        for verse in self.verses:
            total_weight = 0
            # Initiate a list with the weight of each word in this verse. 
            words = [w for w in self.get_vs_words(verse) 
                    if w not in c.stop_words]
            weights = [F.freq_lex.v(wg) for wg in words]
            # Add the scaled word weights to the verse's total weight.
            for weight in weights:
                for i in c.scale.keys():
                    if weight in c.scale[i]:
                        total_weight += i
            # Add the verse weight to the dictionary at this verse's key. 
            total_weight /= len(weights)
            verse_weights[verse] = round(total_weight, 4)
        
        return verse_weights

    def get_weight(self):
        verse_weights = self.get_vs_weights()
        denom = 1 if len(self.verses) < 1 else len(self.verses)
        weight = sum(verse_weights.values()) / denom

        return round(weight, 4)

In [5]:
# Format inputs.
def get_node(reference):

    # Formatting for series of books.
    if reference[0] in ['1', '2']:
        reference = reference.replace(reference[1], '_', 1)
    
    # Split reference into book, chapter, and verse. 
    ref = reference.split()
    bk_ref = ref[0].lower()
    ch_vs = ref[1].split(':')
    ch_ref = int(ch_vs[0])
    vs_ref = int(ch_vs[1])

    # Match bk_ref to the name stored in English book names. 

    # All English OT book names as a space-separated string.
    books = ' '.join([Fs("book@en").v(book) for 
                    book in F.otype.s('book')])
    
    # Formatting for series of books.
    if reference[0] in ['1', '2']:
        bk_ref = bk_ref[:2] + bk_ref[2].upper() + bk_ref[3:]
    else:
        bk_ref = bk_ref.capitalize()

    # Match the abbreviation using regex. 
    bk_ref = re.search(f"{bk_ref}[^\s]*", books).group()
    if not bk_ref:
        print(f"Try one of these: {books}")
        return None
    
    # Get the book's name as it appears in the DB. 
    for book in F.otype.s('book'):
        if bk_ref == Fs("book@en").v(book):
            bk_ref = F.book.v(book)
    
    query = f"""
    book book={bk_ref}
        chapter chapter={ch_ref}
            verse verse={vs_ref}
    """
    result = S.search(textwrap.dedent(query))

    # Return the verse's node value.
    return list(result)[0][-1]

Optimze with [Text Library](https://annotation.github.io/text-fabric/tf/advanced/sections.html#tf.advanced.sections.nodeFromSectionStr)

In [6]:
"""Inputs should be of the format: 'Gen 1:1', 'Gen 3:1'.
   E.g., get_paragraphs('exod 12:1', 'exod 13:1')
   Or use only one argument to get paragraphs from that chapter."""
def get_paragaphs(start_verse, end_verse=None):
    # Maps paragraph numbers to paragraph objects. 
    paragraphs = {}

    # Initiate the counter and store the first paragraph.
    para_num = 1
    paragraphs[para_num] = Paragraph(id=para_num)
    para = paragraphs[para_num]
    para.reset_values()

    # Convert string inputs to verse node values.
    if isinstance(start_verse, int):
        if F.otype.v(start_verse) == 'chapter':
            start_verse = L.d(start_verse, otype='verse')[0]
    else:
        start_verse = get_node(start_verse)

    # Accept inputs with only one argument. Get last verse of that chapter. 
    if not end_verse:
        ch = L.u(start_verse, otype='chapter')
        end_verse = L.d(ch[0], otype='verse')[-1]
    else:
        end_verse = get_node(end_verse)

    verse_range = F.otype.sInterval('verse')
    start_verse -= verse_range[0]
    end_verse -= verse_range[0]
    
    # Iterate through the range of verses passed into the function. 
    for verse in F.otype.s('verse')[start_verse:end_verse]:
        
        # Get the string at the end of the verse. 
        ending = T.text(verse).split()[-1]
        
        # Check if the string is a paragraph marker.  
        if ending in para.types.keys():
            para._type = para.types[ending]
            para.verses.append(verse)

            # Begin a new paragraph. 
            para_num += 1
            paragraphs[para_num] = Paragraph(id=para_num)
            para = paragraphs[para_num]
            para.reset_values()

        else:
            para.verses.append(verse)

    return paragraphs

In [7]:
def order_paragraphs(paragraphs):
    para_list = []
    for para in paragraphs.values():
        para.weight = para.get_weight()
        para_l.append(para)
    para_list.sort(key=lambda p: p.weight)
    return para_list

def passage_weight(paragraphs):
    weight = 0
    verses = 0
    for para in paragraphs.values():
        para.weight = para.get_weight()
        para_weight = para.weight * len(para.verses)
        weight += para_weight
        verses += len(para.verses)
    weight /= verses
    return round(weight, 6)

def all_chapters():
    all_ch_weights = collections.defaultdict(float)
    for chapter in F.otype.s('chapter'):
        bk, ch = T.sectionFromNode(chapter)
        ref = bk + str(ch)
        paragraphs = get_paragaphs(chapter)
        all_ch_weights[ref] = passage_weight(paragraphs)
    return all_ch_weights

In [3]:
paragraphs = get_paragaphs('ex 12:1')
# for para in paragraphs.values():
#     print(f"{para.id} {para.get_text()}\n")

# passages = ['gen 1:1', 'gen 2:1', 'gen 3:1', 'gen 4:1', 'gen 5:1']
# for psg in passages:
#     paragraphs = get_paragaphs(psg)
#     print(f"{psg} - {passage_weight(paragraphs)}")

# for ch in F.otype.s('chapter')[0:10]:
#     paragraphs = get_paragaphs(ch)
#     print(f"{F.chapter.v(ch)} - {passage_weight(paragraphs)}")

        
ordered = order_paragraphs(paragraphs)
for para in ordered:
    print(f"{para.id}\n{para.weight}\n{para.get_text()}")

NameError: name 'get_paragaphs' is not defined

# . . .

In [178]:
all_ch_weights = all_chapters()
all_sorted = {k: v for k, v in sorted(all_ch_weights.items(), key=lambda item: item[1])}
all_sorted

{'Joshua23': 1.4359,
 '1_Chronicles17': 1.451154,
 'Psalms134': 1.4604,
 '2_Samuel7': 1.470057,
 'Ezekiel37': 1.482989,
 'Deuteronomy30': 1.531795,
 'Jeremiah26': 1.537839,
 'Ezekiel2': 1.543578,
 '1_Kings3': 1.555356,
 'Numbers27': 1.571586,
 'Deuteronomy17': 1.576937,
 '2_Chronicles7': 1.595252,
 'Judges2': 1.595841,
 '1_Kings13': 1.618961,
 'Exodus3': 1.6197,
 'Exodus24': 1.620682,
 'Jeremiah42': 1.623976,
 'Deuteronomy6': 1.624833,
 'Joshua4': 1.633704,
 'Jeremiah34': 1.640567,
 '1_Samuel18': 1.641793,
 'Exodus10': 1.644164,
 'Exodus4': 1.645493,
 'Deuteronomy31': 1.648131,
 'Jeremiah44': 1.65489,
 'Genesis39': 1.6619,
 '1_Samuel12': 1.662617,
 'Exodus33': 1.663809,
 'Leviticus17': 1.66512,
 '2_Chronicles23': 1.67018,
 'Deuteronomy26': 1.670883,
 '1_Kings20': 1.67334,
 'Exodus7': 1.676693,
 'Genesis45': 1.6778,
 '1_Kings8': 1.680283,
 'Joshua24': 1.680556,
 '1_Samuel24': 1.681695,
 'Ezekiel11': 1.68525,
 'Ezekiel20': 1.688481,
 'Deuteronomy12': 1.690383,
 'Joshua3': 1.6904,
 'Ezeki

In [5]:
fifty_above = []
hun_above = []
seventy_above = []
forty_above = []
thirty_above = []
for k in all_dict.keys():
    num = (all_dict[k])
    if num >= 100:
        hun_above.append(k)

In [6]:
match = {}
mismatch = {}
for wo in F.otype.s('word'):
    lexemes = [F.lex.v(wo)]
    for lex in lexemes:
        bo, ch, ve = T.sectionFromNode(wo)
        ref = bo + str(ch)
        if lex in hun_above:
            if ref in match: #I didn't use the defaultdict because the next code gave an error saying you couldn’t divide defaultdict
                match[ref] += 1
            else:
                match[ref] = 0
                match[ref] += 1
        else:
            if ref in mismatch:
                mismatch[ref] += 1
            else:
                mismatch[ref] = 0
                mismatch[ref] += 1

In [7]:
d3 = {key: mismatch[key] / match[key] for key in match if key in mismatch}
sorted_d3 = {k: v for k, v in sorted(d3.items(), key=lambda item: item[1])}

In [11]:
sorted_d3

{'Joshua23': 0.045346062052505964,
 '1_Kings3': 0.05116279069767442,
 'Ezekiel37': 0.05272108843537415,
 'Deuteronomy30': 0.05422993492407809,
 '2_Samuel7': 0.05804311774461028,
 '1_Chronicles17': 0.06179775280898876,
 'Deuteronomy17': 0.06391752577319587,
 'Psalms134': 0.06451612903225806,
 'Judges2': 0.06485355648535565,
 'Deuteronomy5': 0.07072368421052631,
 'Jeremiah26': 0.07142857142857142,
 '2_Chronicles1': 0.07142857142857142,
 '2_Chronicles7': 0.0718132854578097,
 'Joshua4': 0.07212475633528265,
 'Exodus11': 0.0730593607305936,
 'Exodus24': 0.07471264367816093,
 'Deuteronomy6': 0.07565011820330969,
 'Jeremiah27': 0.07924528301886792,
 'Deuteronomy12': 0.08018154311649017,
 'Exodus14': 0.08069620253164557,
 'Numbers36': 0.08270676691729323,
 '1_Samuel29': 0.08275862068965517,
 'Numbers27': 0.08353808353808354,
 '1_Samuel9': 0.08358208955223881,
 'Ezekiel20': 0.08415300546448087,
 'Joshua1': 0.08585858585858586,
 'Genesis48': 0.08597285067873303,
 '1_Chronicles22': 0.086330935251